This contains the tools for publishing a public version of first reading resolutions.

This should be used after plenary. 

It will create a folder inside Google Drive labeled 'For campus feedback' with copies of all first reading resolutions

https://developers.google.com/docs/api/quickstart/python

In [ ]:
%cd ~/Dropbox/ResolutionManagerWeb/python-scripts/

# from ResolutionManager import environment as env
from ResolutionManager.API.CredentialsManager import CredentialsManager
from ResolutionManager.Repositories.DocumentRepository import DocumentRepository
from ResolutionManager.Repositories.FileRepository import FileRepository
from ResolutionManager.Repositories.ResolutionTemplateRespository import ResolutionTemplateRepository
from ResolutionManager.Repositories.PermissionsRepository import PermissionsRepository
# import ResolutionManager.executables.make_folders_for_plenary as make_folders
# import ResolutionManager.executables.web_copy_first_readings_for_feedback as copy_first_reads
# import ResolutionManager.executables.create_resolution_from_template as create_resolution
from ResolutionManager.Models.Plenaries import Plenary
from ResolutionManager.Models.Committees import Committee
from ResolutionManager.Models.Resolutions import Resolution
import datetime
# import environment as env


In [ ]:
from ResolutionManager.DAO.DAO import MySqlDao
from ResolutionManager.Repositories.CommitteeRepository import CommitteeRepository
from ResolutionManager.Repositories.PlenaryRepository import PlenaryRepository
from ResolutionManager.Repositories.ResolutionRepository import ResolutionRepository
resolution_id = 11


In [ ]:
dao = MySqlDao()
committee_repo = CommitteeRepository(dao)
plenary_repo = PlenaryRepository(dao)
resolution_repo = ResolutionRepository(dao)
doc_repo = DocumentRepository()
file_repo = FileRepository()

# Get document

In [ ]:
doc_repo = DocumentRepository()

In [ ]:
doc_repo.get_document('1V8wRL5HobjyLLlISGAEvJKbVGaJcsCAwiIgNM_-oikc')

# Named ranges

In [ ]:
# test rez id 
rid = '1XrZTCvSTCOTXnpxGqLAbj0a5EEHuaU_Fo3RdMMYqEqc'
document = doc_repo.get_document(rid)

In [ ]:
document['body']['content']

In [ ]:
document['namedRanges']

In [ ]:
titleRangeId = 'kix.mief0boal1wa'

#create named range

empty segment id = body

'elements': [{'startIndex': 20,
       'endIndex': 52,
       'textRun': {'content': 'The California State University\n',
        'textStyle': {'bold': True,
         'weightedFontFamily': {'fontFamily': 'Atkinson Hyperlegible',
          'weight': 400}}}}],

## Add range name to template

In [ ]:
TITLE_RANGE_NAME = "titleRange"

def add_named_range_to_template(service, template_id, startIndex, endIndex, rangeName=TITLE_RANGE_NAME ):
    change = {
        "createNamedRange" :{
            "name": rangeName,
            "range": {
                "startIndex": startIndex,
                "endIndex": endIndex
            }
        }
    }
    result = service.documents().batchUpdate(
            documentId=template_id, body={'requests': change}).execute()


In [ ]:
template_id = '1V8wRL5HobjyLLlISGAEvJKbVGaJcsCAwiIgNM_-oikc'
add_named_range_to_template(doc_repo.service, template_id, 54, 62)

## Get named range from document

In [ ]:
def get_named_ranges(document, titleRangeName=TITLE_RANGE_NAME):
    # Find the matching named ranges.
    named_range_list = document.get('namedRanges', {}).get(titleRangeName)
    if not named_range_list:
        raise Exception('The named range is no longer present in the document.')

    # Determine all the ranges of text to be removed, and at which indices the
    # replacement text should be inserted.
    all_ranges = []
    insert_at = {}
    for named_range in named_range_list.get('namedRanges'):
        ranges = named_range.get('ranges')

        all_ranges.extend(ranges)
        # Most named ranges only contain one range of text, but it's possible
        # for it to be split into multiple ranges by user edits in the document.
        # The replacement text should only be inserted at the start of the first
        # range.
        insert_at[ranges[0].get('startIndex')] = True

    return all_ranges

In [ ]:
#no longer template
t = doc_repo.get_document('1eYxyexBr7YtdPP5704hqQgoDEa5v9a5VzPcvkIed-ZY')

In [ ]:
template['namedRanges']

In [ ]:
get_named_ranges(template)

In [ ]:
newdoc = doc_repo.get_document('1MlgjsHsGQ9k6SVOAx088sz8FjWFMzw0ZXYJMeIf7NeA')
get_named_ranges(newdoc)

In [ ]:
newdoc

In [ ]:
# replace title from template

In [ ]:
def replace_named_title_range(service, document_id, range_name, new_text):
    """Replaces the text in existing named ranges."""

    # Determine the length of the replacement text, as UTF-16 code units.
    # https://developers.google.com/docs/api/concepts/structure#start_and_end_index
    new_text_len = len(new_text.encode('utf-16-le')) / 2

    # Fetch the document to determine the current indexes of the named ranges.
    document = service.documents().get(documentId=document_id).execute()

    # Find the matching named ranges.
    named_range_list = document.get('namedRanges', {}).get(range_name)
    if not named_range_list:
        raise Exception('The named range is no longer present in the document.')

    # Determine all the ranges of text to be removed, and at which indices the
    # replacement text should be inserted.
    all_ranges = []
    insert_at = {}
    for named_range in named_range_list.get('namedRanges'):
        ranges = named_range.get('ranges')
        all_ranges.extend(ranges)
        # Most named ranges only contain one range of text, but it's possible
        # for it to be split into multiple ranges by user edits in the document.
        # The replacement text should only be inserted at the start of the first
        # range.
        insert_at[ranges[0].get('startIndex')] = True

    # Sort the list of ranges by startIndex, in descending order.
    all_ranges.sort(key=lambda r: r.get('startIndex'), reverse=True)

    # Create a sequence of requests for each range.
    requests = []
    for r in all_ranges:
        # Delete all the content in the existing range.
        requests.append({
            'deleteContentRange': {
                'range': r
            }
        })

        segment_id = r.get('segmentId')
        start = r.get('startIndex')
        if insert_at[start]:
            # Insert the replacement text.
            requests.append({
                'insertText': {
                    'location': {
                        'segmentId': segment_id,
                        'index': start
                    },
                    'text': new_text
                }
            })
            # Re-create the named range on the new text.
            requests.append({
                'createNamedRange': {
                    'name': range_name,
                    'range': {
                        'segmentId': segment_id,
                        'startIndex': start,
                        'endIndex': start + new_text_len
                    }
                }
            })

    # Make a batchUpdate request to apply the changes, ensuring the document
    # hasn't changed since we fetched it.
    body = {
        'requests': requests,
        'writeControl': {
            'requiredRevisionId': document.get('revisionId')
        }
    }
    service.documents().batchUpdate(documentId=document_id, body=body).execute()



In [ ]:
copy_id = '1eYxyexBr7YtdPP5704hqQgoDEa5v9a5VzPcvkIed-ZY'
newTitle = "Dogs are good smelling"

In [ ]:
replace_named_range(doc_repo.service, copy_id, TITLE_RANGE_NAME, newTitle)


## Read title

In [ ]:




def get_title(document, startIndex):
    doc_content = document.get('body').get('content')
    title = ""
    for i in doc_content:
        try:
            if i['startIndex'] == startIndex:
                for e in i['paragraph']['elements']:
                    print(e['textRun']['content'])
                    title += e['textRun']['content']

        except KeyError:
            pass
    title = title.strip().replace('\n', '')
    return title



In [ ]:
docid = "16Bwqhrn38yU1KATbZ1T3B_5ATMMfXiFgKuqXeUKZlwE"
newdoc = doc_repo.get_document(docid)
startIndex = get_named_ranges(newdoc)[0]['startIndex']
get_title(newdoc, startIndex)


In [ ]:
# resolution = resolution_repo.load_resolution(28)

docid = "16Bwqhrn38yU1KATbZ1T3B_5ATMMfXiFgKuqXeUKZlwE"
newdoc = doc_repo.get_document(docid)

resolution_repo.get_named_ranges(newdoc)

In [ ]:
resolution = resolution_repo.load_resolution(30)
resolution.document_id
resolution_repo.get_current_title_from_drive(resolution)

### Sync title w db

In [ ]:
resolution = resolution_repo.load_resolution(30)

resolution_repo.update_title_from_drive_version(resolution)

# Create agenda

In [ ]:
resolutions = []
query = f"select id from ascsu.resolutions"
results = resolution_repo.dao.conn.execute(query)
for r in results:
    rid = r[0]
    sponsor = committee_repo.load_sponsor(rid)
    cosponsors = committee_repo.load_cosponsors(rid)
    rez = resolution_repo.load_resolution(rid, sponsor, cosponsors)
    resolutions.append(rez)
resolutions 
# [Resolution(**r) for r in results]

In [ ]:
resolutions[0].__dict__

In [ ]:
resolutions[0].url

In [ ]:
resolutions[0].agenda_item

In [ ]:
plenary_id = 1

# Load from database
dao = MySqlDao()
committee_repo = CommitteeRepository(dao)
resolution_repo = ResolutionRepository(dao)
plenary_repo = PlenaryRepository(dao)
permission_repo = PermissionsRepository()
document_repo = DocumentRepository()

plenary = plenary_repo.load_plenary(plenary_id)
plenary.__dict__

In [ ]:
plenary_id = 1

# Load from database
dao = MySqlDao()
committee_repo = CommitteeRepository(dao)
resolution_repo = ResolutionRepository(dao)
plenary_repo = PlenaryRepository(dao)
permission_repo = PermissionsRepository()
document_repo = DocumentRepository()

plenary = plenary_repo.load_plenary(plenary_id)

# Get all resolutions
# For now not going to sync the database, just use title from drive if can be retrieved and default to db version
resolutions = resolution_repo.load_all_resolutions()

# Create an agenda document
fname = f"{plenary.plenary_folder_name} Agenda"
agenda_id = document_repo.create_file(fname)
file_repo.move_file_to_folder(agenda_id, plenary.plenary_folder_id)

# maybe should store, that way can update
print(agenda_id)

idx = 1
requests = []

text = ""
try:
    for r in resolutions:
        text += f"{r.agenda_item} \n" 
#         text += f"{r.url}\n"
except Exception:
    print(f"Error with : {r.__dict__}")
    pass

requests.append(    {
        'insertText': {
            'location': {
                'index': idx,
            },
            'text': text,
        }
        })
        

# for r in resolutions:
#     try:
#         text = f"{r.agenda_item} \n"
#         requests.append(    {
#         'insertText': {
#             'location': {
#                 'index': idx,
#             },
#             'text': text,
#         }
#         })
#         idx += len(text)
    
        
#         requests.append(    {
#         'insertText': {
#             'location': {
#                 'index': idx,
#             },
#             'text': r.url,
#         }
#         })
        
# #         requests.append(
# #          {
# #              "updateTextStyle": {
# #                  "textStyle": {
# #                      "link": {
# #                          "url": r.url
# #                      }
# #                  },
# #                  "range": {
# #                      "startIndex": idx,
# #                      "endIndex": idx + len(r.url)
# #                  },
# #                  "fields": "link"
# #              }
# #          }
# #         )
#         idx += len(r.url)

#     except Exception:
#         pass
    
# text = [f"{r.agenda_item} \n" for r in resolutions]
print(requests)
# requests = [
#     {
#         'insertText': {
#             'location': {
#                 'index': 1,
#             },
#             'text': text,
#             'url' : resolutions[0].url

#         }
#     }
# ]
result = document_repo.service.documents().batchUpdate(documentId=agenda_id, body={'requests': requests}).execute()


In [ ]:
committee_repo.load_sponsor(30)

In [ ]:
resolution.__dict__

In [ ]:
p = plenary_repo.load_plenary(1)

In [ ]:
p.month

In [ ]:
p.formatted_plenary_date()

In [ ]:
p.year

In [ ]:
d = datetime.date(2023, 5, 7)
d.day +1

In [ ]:
d.strftime("%B")

In [ ]:
thursday_date = datetime.date(2023, 5, 7)
dao.conn.execute(f"insert into ascsu.plenaries (thursday_date) values ('{thursday_date}')")

In [ ]:
r = dao.conn.execute("select * from ascsu.plenaries").fetchone()
r

In [ ]:
import os
cwd = os.getcwd()
cwd


In [ ]:
resolution_name = "Opposing the existence of the CO"
resolution_number = 3456
committee = Committee('Faculty Affairs', 'FA')
cosponsors = [ Committee('Academic Affairs', 'AA')]
resolution = Resolution(number=resolution_number, title=resolution_name, committee=committee, cosponsors=cosponsors)

plenary = Plenary(thursday_date='2023-01-04',
                      first_reading_folder_id='1sv_4BUV5fk6Kcjss8HeSCJWnsLZJVpKC',
                  plenary_folder_id='1ITs5N1qpTbqVqAhALrxqSsDwiSKsnSj5'
                      )
# plenary = Plenary(year=2023,
#                       month='September',
#                       thursday_date=12,
#                       friday_date=14,
#                       first_reading_folder_id='1sv_4BUV5fk6Kcjss8HeSCJWnsLZJVpKC',
#                   plenary_folder_id='1ITs5N1qpTbqVqAhALrxqSsDwiSKsnSj5'
#                       )

In [ ]:
doc_repo = DocumentRepository()
file_repo = FileRepository()


In [ ]:
committees = [ ('Academic Affairs', 'AA'), ('Academic Prep', 'APEP'), 
              ('Executive Committee', 'Exec'),('Faculty Affairs', 'FA'), 
              ('Fiscal and Governmental Affairs', 'FGA'), ('Justice, Equity, Diversity, and Inclusion', 'JEDI')]
committees = [Committee(c[0], c[1]) for c in committees]

In [ ]:
k = file_repo.list_files(folder_id=plenary.first_reading_folder_id)


In [ ]:
k

# Build folder structure for plenaries

In [ ]:
make_folders.main(plenary)

In [ ]:
plenary2 = Plenary(thursday_date='2023-01-23')
plenary2 = make_folders.main(plenary2)
plenary2.__dict__

# Create resolution

In [ ]:
plenary = Plenary(id=1, thursday_date='2023-01-23')


resolution_name = "Opposing the existence of the CO"
resolution_number = 3456
committee = Committee('Faculty Affairs', 'FA')
cosponsors = [ Committee('Academic Affairs', 'AA')]
resolution = Resolution(number=resolution_number, title=resolution_name, committee=committee, cosponsors=cosponsors)

create_resolution.main(plenary_id=plenary.id, resolution_id=1)

In [ ]:
%cd ~/Dropbox/ResolutionManagerWeb/python-scripts/executables/
import web_create_resolution_from_template as create_resolution


# Create public feedback versions

In [ ]:
copy_first_reads.main(plenary)

In [ ]:
copy_first_reads.main(1)

# Permission management

In [ ]:
fid = '1tYVZaK0Naaekhj6f4phj7xF4XVIIn5bUF6aJI6SKSZM' #test 8
# fid = '1sv_4BUV5fk6Kcjss8HeSCJWnsLZJVpKC' #first read folder
permission_repo = PermissionsRepository()


In [ ]:
# make editable
for f in file_repo.list_files(folder_id=plenary.first_reading_folder_id):
    permission_repo.make_world_writable(f['id'])

In [ ]:
#make readable
for f in file_repo.list_files(folder_id=plenary.first_reading_folder_id):
    permission_repo.make_world_readable(f['id'])

In [ ]:
permission_repo.remove_world_readable(fid)

In [ ]:
permission_repo.get_permission(fid)

In [ ]:
permission_repo.get_sharing_link(fid)

# display

In [ ]:
from ResolutionManager.Widgets.InputFields import make_text_input, make_selection_button
from IPython.display import display
from ipywidgets import Layout, widgets, HBox, VBox, Label


In [ ]:
class ControlStore(object):
    
    def __init__(self):
        self.resolution = Resolution()
        
    @property
    def sponsor(self):
        return self.resolution.committee
    
    @sponsor.setter
    def sponsor(self, committee):
        self.resolution.committee = committee
    
    @property
    def cosponsors(self):
        return self.resolution.cosponsors
    
    @cosponsors.setter
    def cosponsors(self, cosponsors):
        self.resolution.cosponsors = cosponsors
            
    @property
    def title(self):
        return self.resolution.title
    
    @title.setter
    def title(self, title):
        self.resolution.title = title

    def get_sponsor(self):
        return [self.sponsor]

    def add_sponsor(self, committee, kwargs):
        print('add')
        print(committee)
        self.sponsor = committee
          
    def remove_sponsor(self, committee):
        print('remove')
        print(committee)
        self.sponsor =  None
        
        
    def get_cosponsors(self):
        return self.cosponsors
    
    def add_cosponsors(self, co):
        self.cosponsors = co
        
    def remove_cosponsors(self, c):
        self.cosponsors = []
    
store = ControlStore()

In [ ]:
# sponsoring_committee = None
# def get_sponsor():
#     return [sponsoring_committee]

# def add_sponsor(committee, kwargs):
#     sponsoring_committee = committee, kwargs
    
# def remove_sponsor(committee):
#     print(committee)
#     sponsoring_committee = None

def make_committee_chooser():
    buttons = []
    for c in committees:
        v = {'item_id' : c, 'name' : c.name, 'get_func' : store.get_sponsor, 'add_func': store.add_sponsor, 'remove_func' : store.remove_sponsor}
        buttons.append(make_selection_button(**v))
    return buttons

def make_cosponsor_chooser():
    buttons = []
    for c in committees:
        v = {'item_id' : c, 'name' : c.name, 'get_func' : store.get_cosponsors, 'add_func': store.add_cosponsors, 'remove_func' : store.remove_cosponsors}
        buttons.append(make_selection_button(**v))
    return buttons

def make_waiver():
    return widgets.Checkbox(
    value=False,
    description='Waiver requested',
    disabled=False,
    indent=False
)

def make_create_button():
    def callback():
        create_resolution.main(plenary=plenary, resolution=store.resolution)
        print('click')
    layout = widgets.Layout( width='75%' )
    b = widgets.Button( description="Create resolution", layout=layout, button_style='danger' )
    b.on_click(callback)
    return b

def make_populate_folders_button():
    pass


In [ ]:
row_layout = Layout( width='100%', padding='10px' )
row1 = HBox([Label("Resolution title"), make_text_input({'label': 'Resolution title', 'handler': store.title})], layout=row_layout)
row2 = HBox([Label("Sponsoring committee")], layout=row_layout)
row3 = VBox(make_committee_chooser(), layout=row_layout)
row4 = HBox([Label("Cosponsoring committees")], layout=row_layout)
row5 = VBox(make_cosponsor_chooser(), layout=row_layout)
row6 = HBox([make_waiver()], layout=row_layout)
row7 = VBox([make_create_button()], layout=row_layout)

container = VBox([row1, row2, row3, row4, row5, row6, row7], layout=Layout(border='dashed', width='100%'))
display(container)


In [ ]:
make_action_button()

In [ ]:
display(make_action_button())

In [ ]:
beep = False
def callback(change):
    beep =True
    print('click')
layout = widgets.Layout( width='100%' )
b = widgets.Button( description="Create resolution", layout=layout, button_style='danger' )
b.on_click(callback)
display(b)    

In [ ]:
def make_selection_button( item_id, name, get_func, add_func, remove_func, width='50%', **kwargs ):


In [ ]:
beep

In [ ]:
c.sponsor